In [1]:
from pyspark.sql.functions import *

Starting Spark application


ID,YARN Application ID,Kind,State,Spark UI,Driver log,Current session?
22,application_1631311336051_0040,pyspark,idle,Link,Link,✔


FloatProgress(value=0.0, bar_style='info', description='Progress:', layout=Layout(height='25px', width='50%'),…

SparkSession available as 'spark'.


FloatProgress(value=0.0, bar_style='info', description='Progress:', layout=Layout(height='25px', width='50%'),…

In [2]:
# Application sessions records for SSR
ssr = spark.read.parquet("s3://mist-secorapp-staging/session-stats-analytics/session-stats-analytics-staging/dt=2021-09-21/hr=*/");
print("Total session records count is : {}".format(ssr.count()))

FloatProgress(value=0.0, bar_style='info', description='Progress:', layout=Layout(height='25px', width='50%'),…

Total session records count is : 1750862

In [4]:
org_id = 'cb768f63-1f32-455c-acf7-d98a7e579dd0'

# Different sites and chassis_mac under the given org
print("Given org_id, see all site_id and chassis_mac")
dataForOrg = ssr.filter(col('org_id') == org_id)
dataForOrg.select('chassis_mac', 'site_id', 'org_id').distinct().show(10, truncate=False)
print("\n")

FloatProgress(value=0.0, bar_style='info', description='Progress:', layout=Layout(height='25px', width='50%'),…

Given org_id, see all site_id and chassis_mac
+------------+------------------------------------+------------------------------------+
|chassis_mac |site_id                             |org_id                              |
+------------+------------------------------------+------------------------------------+
|0200016c2c82|5a981720-55e5-498c-899c-2586c4dc82c9|cb768f63-1f32-455c-acf7-d98a7e579dd0|
|02000196a63a|d990a812-be9e-40aa-9837-ccf944e280fc|cb768f63-1f32-455c-acf7-d98a7e579dd0|
|020001108266|0a84a119-6766-49ae-a439-f46a040e9182|cb768f63-1f32-455c-acf7-d98a7e579dd0|
|02000138d539|5a981720-55e5-498c-899c-2586c4dc82c9|cb768f63-1f32-455c-acf7-d98a7e579dd0|
|0200011a63b7|a793ad34-d411-4584-82e0-3ad37665c080|cb768f63-1f32-455c-acf7-d98a7e579dd0|
|020001e4fd4d|d990a812-be9e-40aa-9837-ccf944e280fc|cb768f63-1f32-455c-acf7-d98a7e579dd0|
|0200017a48fa|0a84a119-6766-49ae-a439-f46a040e9182|cb768f63-1f32-455c-acf7-d98a7e579dd0|
|0200010bd3cf|3bf704d8-a0a6-416b-9623-5bcc2ed042bb|cb768f63-1f32

In [11]:
site_id = '0a84a119-6766-49ae-a439-f46a040e9182'
chassis_mac = '020001108266'

dataForSite = dataForOrg.filter(col('site_id') == site_id)
print("Total session records for org_id: {} and site_id: {} is {}".format(org_id, site_id, dataForSite.count()))
print("\n")
dataForSite.cache()

dataForChassisMac = dataForOrg.filter(col('chassis_mac') == chassis_mac)
print("Total session records for org_id: {}, site_id: {} and chassis_mac: {} is {}".format(org_id, site_id, chassis_mac, dataForChassisMac.count()))
print("\n")
dataForChassisMac.cache()

print("For the given org_id: {}, site_id: {} and chassis_mac: {} following are stats".format(org_id, site_id, chassis_mac))
print("\n")

FloatProgress(value=0.0, bar_style='info', description='Progress:', layout=Layout(height='25px', width='50%'),…

Total session records for org_id: cb768f63-1f32-455c-acf7-d98a7e579dd0 and site_id: 0a84a119-6766-49ae-a439-f46a040e9182 is 602641


Total session records for org_id: cb768f63-1f32-455c-acf7-d98a7e579dd0, site_id: 0a84a119-6766-49ae-a439-f46a040e9182 and chassis_mac: 020001108266 is 302599


For the given org_id: cb768f63-1f32-455c-acf7-d98a7e579dd0, site_id: 0a84a119-6766-49ae-a439-f46a040e9182 and chassis_mac: 020001108266 following are stats

In [7]:
print("See all distinct applications and session record counts")
dataForSite.groupBy('application').count().distinct().sort(desc('count')).show(40, truncate=False)
print("\n")

FloatProgress(value=0.0, bar_style='info', description='Progress:', layout=Layout(height='25px', width='50%'),…

See all distinct applications and session record counts
+-----------+--------------+------+
|application|normalized_app|count |
+-----------+--------------+------+
|           |              |602641|
+-----------+--------------+------+

In [8]:
print("Number of records group by event_type, direction and next_hop_type")
dataForChassisMac.groupBy('event_type', 'direction', 'next_hop_type').agg(count('*')).sort(desc('count(1)')).show()
print("\n")

FloatProgress(value=0.0, bar_style='info', description='Progress:', layout=Layout(height='25px', width='50%'),…

Number of records group by event_type, direction and next_hop_type
+----------+---------+-------------+--------+
|event_type|direction|next_hop_type|count(1)|
+----------+---------+-------------+--------+
|         3|        2|            1|  139196|
|         3|        1|            1|  139191|
|         2|        1|            1|   39200|
|         2|        2|            1|   39200|
|         4|        1|            0|    7823|
+----------+---------+-------------+--------+

In [9]:
print("Statistics for records considering event_type 3 or 5, direction 1, next_hop_type 1 or 3, session_id not empty")
dataForChassisMac.filter(((col('event_type') == 3) | (col('event_type') == 5)) & (col("direction") == 1) & (col("session_id") != '') & ((col("next_hop_type") == 1) | (col("next_hop_type") == 3))).withColumn("total_session_time_in_secs", (col('flow_end_time')-col('flow_start_time'))/1000000).select(((col('bytes_from_client') * 0.0078)/col('total_session_time_in_secs').cast('int')).alias('kbps_from_client'),col('total_session_time_in_secs')).summary("count","mean", "stddev","min", "max", "75%", "90%", "95%", "99%").withColumn('kbps_from_client', round(col('kbps_from_client'),2)).withColumn('total_session_time_in_secs', round(col('total_session_time_in_secs'),2)).show()
print("\n")

FloatProgress(value=0.0, bar_style='info', description='Progress:', layout=Layout(height='25px', width='50%'),…

Statistics for records considering event_type 3 or 5, direction 1, next_hop_type 1 or 3, session_id not empty
+-------+----------------+--------------------------+
|summary|kbps_from_client|total_session_time_in_secs|
+-------+----------------+--------------------------+
|  count|        139191.0|                  139191.0|
|   mean|            3.99|                     13.84|
| stddev|            1.12|                    169.88|
|    min|             0.0|                       5.0|
|    max|          167.55|                   16846.0|
|    75%|            4.44|                       6.0|
|    90%|            4.44|                       6.0|
|    95%|            4.44|                       6.0|
|    99%|            5.37|                     176.0|
+-------+----------------+--------------------------+

In [12]:
print("Daywise Teams call with given application name and all relevent filters and kbps_from_client >= 10 and total_session_time_in_secs >= 180")
dataForChassisMac.filter((col('direction') == 1) & ((col('event_type') == 3) | (col('event_type') == 5)) & (col("session_id") != '') & ((col("next_hop_type") == 1) | (col("next_hop_type") == 3)) & ((col('application')=='') | (col('application')=='OFFICE'))).withColumn("total_session_time_in_secs", (col('flow_end_time')-col('flow_start_time'))/1000000).select(date_format(from_utc_timestamp(from_unixtime(col('flow_start_time')/1000000),"America/New_York"), 'EEEE').alias('day'), from_utc_timestamp(from_unixtime(col("flow_start_time")/1000000, 'yyyy-MM-dd HH:mm'), "America/New_York").alias("start_time_EDT"), from_utc_timestamp(from_unixtime(col("flow_end_time")/1000000, 'yyyy-MM-dd HH:mm'), "America/New_York").alias("end_time_EDT"), col('total_session_time_in_secs').cast('int').alias('total_time'), ((col('bytes_from_client') * 0.0078)/col('total_session_time_in_secs').cast('int')).alias('kbps_from_client')).filter((col('kbps_from_client') >= 10) & (col('total_time') >= 180)).groupBy('day', 'start_time_EDT', 'end_time_EDT').agg(avg('total_time').cast('int').alias('total_time'), avg('kbps_from_client').cast('int').alias('kbps_from_client')).sort('start_time_EDT').show(50, truncate=False)
print("\n")

FloatProgress(value=0.0, bar_style='info', description='Progress:', layout=Layout(height='25px', width='50%'),…

Daywise Teams call with given application name and all relevent filters and kbps_from_client >= 10 and total_session_time_in_secs >= 180
+-------+-------------------+-------------------+----------+----------------+
|day    |start_time_EDT     |end_time_EDT       |total_time|kbps_from_client|
+-------+-------------------+-------------------+----------+----------------+
|Monday |2021-09-20 23:11:00|2021-09-21 00:54:00|6186      |42              |
|Tuesday|2021-09-21 01:38:00|2021-09-21 06:00:00|15750     |35              |
|Tuesday|2021-09-21 01:59:00|2021-09-21 02:09:00|580       |15              |
|Tuesday|2021-09-21 01:59:00|2021-09-21 02:06:00|402       |21              |
|Tuesday|2021-09-21 06:01:00|2021-09-21 11:05:00|18243     |34              |
|Tuesday|2021-09-21 12:59:00|2021-09-21 13:43:00|2614      |21              |
|Tuesday|2021-09-21 13:10:00|2021-09-21 13:51:00|2444      |93              |
|Tuesday|2021-09-21 13:19:00|2021-09-21 13:58:00|2367      |16              |
|Tues

In [15]:
print("All the relevent session records with applicable filters and kbps_from_client >= 10 and total_session_time_in_secs >= 180")
dataForChassisMac.filter((col("direction") == 1) & (col("session_id") != '') & ((col("next_hop_type") == 1) | (col("next_hop_type") == 3)) & ((col("event_type") == 3)| (col("event_type") == 5))).withColumn("total_session_time_in_secs", (col("flow_end_time")-col("flow_start_time"))/1000000).select("application", "session_id", "next_hop_type","event_type", from_unixtime(col("flow_end_time")/1000000).alias("flow_end_time"), from_unixtime(col("flow_start_time")/1000000).alias("flow_start_time"), col("total_session_time_in_secs").cast("int").alias("total_time"),"src_addr", "dst_addr", "src_port", "dst_port", "tenant", "protocol_id", "service_class", "service_name", ((col("bytes_from_client") * 0.0078)/col("total_session_time_in_secs").cast("int")).alias("kbps_from_client"), "pkts_from_client", ((col("bytes_from_server") * 0.0078)/col("total_session_time_in_secs").cast("int")).alias("kbps_from_server"), "pkts_from_server", "client_wcid").filter((col("kbps_from_client")>=10) & (col("total_session_time_in_secs")>=180)).sort(asc("flow_start_time"), "session_id").show(50, truncate=False)
print("\n")

FloatProgress(value=0.0, bar_style='info', description='Progress:', layout=Layout(height='25px', width='50%'),…

All the relevent session records with applicable filters and kbps_from_client >= 10 and total_session_time_in_secs >= 180
+-----------+------------------------------------+-------------+----------+-------------------+-------------------+----------+-------------+--------------+--------+--------+------+-----------+--------------+------------+------------------+----------------+----------------+----------------+------------------------------------+
|application|session_id                          |next_hop_type|event_type|flow_end_time      |flow_start_time    |total_time|src_addr     |dst_addr      |src_port|dst_port|tenant|protocol_id|service_class |service_name|kbps_from_client  |pkts_from_client|kbps_from_server|pkts_from_server|client_wcid                         |
+-----------+------------------------------------+-------------+----------+-------------------+-------------------+----------+-------------+--------------+--------+--------+------+-----------+--------------+------------+--